In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
# /Users/chriswesterman/Library/Jupyter/nbextensions/snippets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import beatthebookies.data as btb

pd.set_option('display.max_columns', 500)


# Exploratory notebook

In [126]:
df = btb.get_data()
# test2 = btb.get_data(season=('2008/2009', '2009/2010'))
# test = pd.concat([test, test2], axis=0, ignore_index=True)

get_data 8.5


In [128]:
df.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_home_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,away_t_away_goals,away_t_away_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,home_t_away_goals,home_t_away_goals_against,home_t_away_wins,home_t_away_losses,home_t_away_draws,away_t_home_goals,away_t_home_goals_against,away_t_home_wins,away_t_home_losses,away_t_home_draws
312,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,30,5,8,5,13,34,3,13,2,24,25,5,5,9,26,30,6,8,5
311,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,29,5,5,8,32,17,11,2,5,27,23,7,7,5,35,18,12,1,6
310,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,11,11,3,4,13,37,1,14,3,34,25,8,4,7,14,35,2,12,5
315,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,21,10,5,3,19,24,5,7,6,20,19,7,6,6,19,23,6,10,3
313,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,8,11,2,5,21,30,6,8,4,22,26,7,8,4,23,34,5,9,5


In [123]:
df['home_w'] = 0

In [111]:
df[['home_t_total_goals','away_t_total_goals']] = 0
    

In [109]:
df['home_t_total_goals'] = df.apply(lambda x: opposite_stat(x, team='home_team', stat='away_team_goal' ), axis=1)
df['away_t_total_goals'] = df.apply(lambda x: opposite_stat(x, team='away_team', stat='home_team_goal' ), axis=1)

In [110]:
df.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,draw,home_t_home_goals,home_t_total_goals,home_t_home_goals_against,home_t_total_goals_against,home_t_home_wins,home_t_home_losses,home_t_home_draws,home_t_total_wins,home_t_total_losses,home_t_total_draws,away_t_away_goals,away_t_total_goals,away_t_away_goals_against,away_t_total_goals_against,away_t_away_wins,away_t_away_losses,away_t_away_draws,away_t_total_wins,away_t_total_losses,away_t_total_draws
312,4701,2015/2016,38,2016-05-15 00:00:00,Everton,Norwich City,3,0,1,0,0,32,24,30,0,5,8,5,0,0,0,13,26,34,0,3,13,2,0,0,0
311,4700,2015/2016,38,2016-05-15 00:00:00,Chelsea,Leicester City,1,1,0,0,1,31,27,29,0,5,5,8,0,0,0,32,35,17,0,11,2,5,0,0,0
310,4699,2015/2016,38,2016-05-15 00:00:00,Arsenal,Aston Villa,4,0,1,0,0,27,34,11,0,11,3,4,0,0,0,13,14,37,0,1,14,3,0,0,0
315,4704,2015/2016,38,2016-05-15 00:00:00,Southampton,Crystal Palace,4,1,1,0,0,35,20,21,0,10,5,3,0,0,0,19,19,24,0,5,7,6,0,0,0
313,4702,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1,1,0,0,24,22,8,0,11,2,5,0,0,0,21,23,30,0,6,8,4,0,0,0


In [38]:

test['home_t_home_goals'] = test.groupby('home_team')['home_team_goal'].apply(lambda x  : x.cumsum().shift(fill_value=0))


In [40]:
test.tail()

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_t_home_goals
755,2426,2009/2010,38,2010-05-09 00:00:00,Manchester United,Stoke City,4,0,91
756,2427,2009/2010,38,2010-05-09 00:00:00,West Ham United,Manchester City,1,1,52
757,2428,2009/2010,38,2010-05-09 00:00:00,Hull City,Liverpool,0,0,40
758,2421,2009/2010,38,2010-05-09 00:00:00,Chelsea,Wigan Athletic,8,0,93
759,2420,2009/2010,38,2010-05-09 00:00:00,Everton,Portsmouth,1,0,65


In [ ]:
[['A',1,1],
['A',2,2],
['A',1,3],
['A',2,4],
['A',1,5],
['B',1,6],
['B',2,7],
['B',1,8],
['B',2,9],
['B',1,10]]

In [49]:
mock = pd.DataFrame([['A',1,1],
                    ['A',2,2],
                    ['A',1,3],
                    ['A',2,4],
                    ['A',1,5],
                    ['B',1,6],
                    ['B',2,7],
                    ['B',1,8],
                    ['B',2,9],
                    ['B',1,10]], columns=['year','team','score'])

In [22]:
mock.head(10)

,year,team,score
0,A,1,1
1,A,2,2
2,A,1,3
3,A,2,4
4,A,1,5
5,B,1,6
6,B,2,7
7,B,1,8
8,B,2,9
9,B,1,10


In [52]:
mock['score_agg'] = mock.groupby('year').apply(pd.groupby('team')['score'].apply(lambda x  : x.cumsum().shift(fill_value=0)))

AttributeError: module 'pandas' has no attribute 'groupby'

In [45]:
mock

,year,team,score
0,A,1,1
1,A,2,2
2,A,1,3
3,A,2,4
4,A,1,5
5,B,1,6
6,B,2,7
7,B,1,8
8,B,2,9
9,B,1,10


In [53]:
mock['score_agg'] = mock.groupby(['year','team'])['score'].apply(lambda x  : x.cumsum().shift(fill_value=0))

In [54]:
mock

,year,team,score,score_agg
0,A,1,1,0
1,A,2,2,0
2,A,1,3,1
3,A,2,4,2
4,A,1,5,4
5,B,1,6,0
6,B,2,7,0
7,B,1,8,6
8,B,2,9,7
9,B,1,10,14
